In [190]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob
from collections import OrderedDict

ddir = "data"

dgwas=pd.read_csv('/data/analysis/UKBB/raw/I10.gwas.imputed_v3.both_sexes.tsv',sep='\t')
phenotype=pd.read_csv('/data/analysis/UKBB/raw/phenotypes.both_sexes.tsv.gz',sep='\t')

In [202]:
#Use only high confident variant
dgwas_sub = dgwas.loc[~dgwas['low_confidence_variant'],:]
snp = pd.DataFrame(dgwas_sub.variant.str.split(':',0).tolist(),columns = ['chr','pos', 'ref', 'alt'])
dgwas_sub_snp = pd.concat([snp, dgwas_sub.reset_index()], axis=1)


In [203]:
#use snps in chr 1-22
dgwas_sub_snp_chr = dgwas_sub_snp[dgwas_sub_snp['chr'].isin(list(map(lambda x: str(x), range(1,23))))]

In [204]:
print(dgwas_sub_snp_chr['chr'].unique())
print(dgwas_sub_snp_chr.head())
dgwas_sub_snp_chr.chr = dgwas_sub_snp_chr.chr.astype(int)
dgwas_sub_snp_chr.pos = dgwas_sub_snp_chr.pos.astype(int)

['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22']
  chr     pos ref alt  index        variant minor_allele  minor_AF  \
0   1  692794  CA   C      4  1:692794:CA:C            C  0.110640   
1   1  693731   A   G      5   1:693731:A:G            G  0.115830   
2   1  707522   G   C      6   1:707522:G:C            C  0.097303   
3   1  717587   G   A      7   1:717587:G:A            A  0.015688   
4   1  730087   T   C      9   1:730087:T:C            C  0.056460   

   expected_case_minor_AC  low_confidence_variant  n_complete_samples  \
0                191.4070                   False              361194   
1                200.3860                   False              361194   
2                168.3350                   False              361194   
3                 27.1403                   False              361194   
4                 97.6761                   False              361194   

        AC       ytx      beta      

/data/env/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [206]:
dgwas_min = dgwas_sub_snp_chr.nsmallest(100, 'pval')
dgwas_min = dgwas_min.sort_values(['chr', 'pos'], ascending=[True, True])

In [209]:
#select interesting regions
snps = OrderedDict()
for index, row in dgwas_min.iterrows():
    chrs = row['chr']
    pos = row['pos']
    if chrs in snps:
        if any(list(map(lambda x: x[0] <= pos <= x[1], snps[chrs]))):
            continue
        else:
            snps[chrs].append((pos-1000000, pos+1000000))
    else:
        snps[chrs]= [(int(pos)-1000000, int(pos)+1000000)]


odict_keys([3, 4, 6, 7, 8, 9, 10, 11, 12, 15, 17])


In [292]:
#get snps in the interesting regions
n = 1
for chrs in snps: 
    print("processing chr",chrs)
    for region in snps[chrs]:
        if n == 1:
            table = dgwas_sub_snp_chr.loc[(dgwas_sub_snp_chr["chr"] == chrs) & ( dgwas_sub_snp_chr["pos"] >= region[0]) & 
       (dgwas_sub_snp_chr["pos"] <= region[1]), ]
            table['SEGNUMBER'] = n
        else:
            temp = dgwas_sub_snp_chr.loc[(dgwas_sub_snp_chr["chr"] == chrs) & ( dgwas_sub_snp_chr["pos"] >= region[0]) & 
       (dgwas_sub_snp_chr["pos"] <= region[1]), ]
            temp['SEGNUMBER'] = n
            table = table.append(temp)
    n = n + 1

processing chr 3
processing chr 4


/data/env/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/data/env/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


processing chr 6
processing chr 7
processing chr 8
processing chr 9
processing chr 10
processing chr 11
processing chr 12
processing chr 15
processing chr 17


In [293]:
table['snp'] = table["chr"].map(str) + "_" + table["pos"].map(str)
print(table.shape)

(107217, 19)


In [301]:
indices = table.groupby('snp')['pval'].idxmin
table_unique = table.loc[indices]

In [319]:
table_unique = table_unique.sort_values(['chr', 'pos'], ascending=[True, True])
table_unique.head()

,chr,pos,ref,alt,index,variant,minor_allele,minor_AF,expected_case_minor_AC,low_confidence_variant,n_complete_samples,AC,ytx,beta,se,tstat,pval,SEGNUMBER,snp
1874926,3,149461748,T,C,2900712,3:149461748:T:C,C,0.060348,104.4020,False,361194,43594.8,101.3690,-0.000055,0.000242,-0.225786,0.821368,1,3_149461748
1874927,3,149462893,T,G,2900713,3:149462893:T:G,G,0.488693,845.4390,False,361194,353026.0,857.9960,0.000078,0.000115,0.674998,0.499678,1,3_149462893
1874928,3,149463939,T,C,2900714,3:149463939:T:C,C,0.017110,29.6006,False,361194,12360.2,26.2549,-0.000318,0.000444,-0.717405,0.473125,1,3_149463939
1874929,3,149464402,CA,C,2900716,3:149464402:CA:C,C,0.494635,855.7190,False,361194,357319.0,879.1730,0.000142,0.000124,1.147810,0.251046,1,3_149464402
1874930,3,149464427,A,G,2900717,3:149464427:A:G,G,0.410530,710.2170,False,361194,296562.0,725.1490,0.000092,0.000117,0.785214,0.432329,1,3_149464427


In [320]:
res = table_unique[['variant', 'chr', 'pos', 'tstat', 'minor_AF', 'SEGNUMBER']]

In [321]:
res = res.rename(columns={'variant': 'SNPID', 'chr': 'CHR', 'pos': 'POS',
                   'tstat': 'Z', 'minor_AF': 'F'})

In [322]:
res['NCONTROL'] = int(phenotype.loc[phenotype['phenotype']=='I10']['n_controls'])
res['NCASE'] = int(phenotype.loc[phenotype['phenotype']=='I10']['n_cases'])

In [323]:
res.to_csv('/data/analysis/UKBB/processed/I10.gwas.imputed_v3.both_sexes.finemapping.tsv.gz',
           sep=' ',header=True, index=False, compression="gzip")
           

In [324]:
res['pheno'] = np.around(np.random.uniform(0,1,res.shape[0]), decimals=3)

In [325]:
res.to_csv('/data/analysis/UKBB/processed/I10.gwas.imputed_v3.both_sexes.finemapping.pheno.tsv.gz',
           sep=' ',header=True, index=False, compression="gzip")